This example demonstrates using SURE to compute metacells for scATAC-seq data. The example data can be downloaded from the [Chan Zuckerberg CELL by GENE Discover](https://cellxgene.cziscience.com/collections/8191c283-0816-424b-9b61-c3e1d6258a77). Users can also download the data using the wget command provided below.

```bash
wget -c https://datasets.cellxgene.cziscience.com/9bc3febe-9043-4781-bb87-b8647c4f2162.h5ad
```

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc

import datatable as dt

from sklearn.preprocessing import OneHotEncoder


/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read_h5ad('9bc3febe-9043-4781-bb87-b8647c4f2162.h5ad')
adata.X = adata.raw.X.copy()
adata.layers['counts'] = adata.X.copy()

#### Select 2000 highly variable genes (HVGs)

In [3]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=True)

/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


#### Save counts, cell names, and HVG names to local files

In [4]:
dt.Frame(pd.DataFrame(adata.layers['counts'].toarray(), columns=adata.var_names, dtype=np.float64)).to_csv('heart_atac_counts.txt.gz')


adata.obs_names.to_frame().to_csv('heart_atac_cell.csv.gz', index=None)
adata.var_names.to_frame().to_csv('heart_atac_gene.csv.gz', index=None)

#### Save undesired factor values to local file

In [5]:
enc = OneHotEncoder(sparse_output=False).fit(adata.obs['donor_id'].to_numpy().reshape(-1,1))
pd.DataFrame(enc.transform(adata.obs['donor_id'].to_numpy().reshape(-1,1)), columns=enc.categories_).to_csv('heart_atac_uwv.txt.gz', index=False)

Different versions of Scikit-learn's OneHotEncoder may have different parameters. Some versions use `sparse`, while others use `sparse_output`. Users need to adjust the parameters according to their own environment.